In [ ]:
## TO DO
Add line to df_lifespan ('FUNCTIONAL_AREA_NODE_3_MODIFIED'), then add longest job for a start date on that line. This should provide the opportunity to put jobs under the longest maintenance period.
### ARIMA
target_variable = df_lifespan['ACTUAL_WORK_IN_MINUTES']

plt.figure(figsize=(10,6))
plt.plot(target_variable)
plt.title('Actual Work Time in Minutes Over Time')
plt.xlabel('Date')
plt.ylabel('Actual Work in Minutes')
plt.show()
from statsmodels.tsa.stattools import adfuller

adf_test = adfuller(target_variable.dropna())
print(f'ADF Statistic: {adf_test[0]}')
print(f'p-value: {adf_test[1]}')
import numpy as np

# Parameters
num_simulations = 100  # Number of Monte Carlo simulations
num_jobs = len(df_lifespan)  # Number of jobs to schedule

# Create a placeholder for the simulation results
sim_results = []

# Monte Carlo simulation loop
for i in range(num_simulations):
    # Simulate the random 'LIFE' and 'MAINTENANCE TIME' for each job
    df_lifespan['SIMULATED_LIFE'] = np.random.normal(df_lifespan['MEAN_LIFESPAN'], 0.2)  # Example: normal distribution with a standard deviation of 0.2
    df_lifespan['SIMULATED_MAINTENANCE_TIME'] = np.random.normal(df_lifespan['MEAN_TIME_MAINTENENCE'], 5)  # Example: normal distribution with a standard deviation of 5
    
    # Simulate start dates based on the simulated life span
    df_lifespan['SIMULATED_START_DATE'] = df_lifespan['EXECUTION_START_DATE'] + pd.to_timedelta(df_lifespan['SIMULATED_LIFE'], unit='D')
    
    # Calculate total downtime (or any other KPI you want to optimize)
    total_maintenance_time = df_lifespan['SIMULATED_MAINTENANCE_TIME'].sum()
    sim_results.append(total_maintenance_time)

# Convert results into a DataFrame
sim_results_df = pd.DataFrame(sim_results, columns=['Total_Maintenance_Time'])

# Analyze the simulation results
optimal_schedule = sim_results_df['Total_Maintenance_Time'].min()  # Example: minimize total maintenance time
optimal_simulation = sim_results_df[sim_results_df['Total_Maintenance_Time'] == optimal_schedule].index[0]

print(f"Optimal total maintenance time: {optimal_schedule}")
print(f"Optimal simulation index: {optimal_simulation}")

# Plot the distribution of total maintenance times
plt.hist(sim_results_df['Total_Maintenance_Time'], bins=50, alpha=0.7, color='blue')
plt.title('Distribution of Total Maintenance Time Across Simulations')
plt.xlabel('Total Maintenance Time')
plt.ylabel('Frequency')
plt.show()
# Define the date range (e.g., simulate the past year)
one_year_ago = pd.Timestamp.today() - pd.DateOffset(years=1)

# Filter the DataFrame to include only jobs from the past year
df_lifespan_filtered = df_lifespan[df_lifespan['EXECUTION_START_DATE'] >= one_year_ago].copy()

# Parameters
num_simulations = 1000  # Number of Monte Carlo simulations
num_jobs = len(df_lifespan_filtered)  # Number of jobs in the past year

# Create a placeholder for the simulation results
sim_results = []

# Monte Carlo simulation loop
for i in range(num_simulations):
    # Simulate the random 'LIFE' and 'MAINTENANCE TIME' for each job in the past year
    df_lifespan_filtered['SIMULATED_LIFE'] = np.random.normal(df_lifespan_filtered['MEAN_LIFESPAN'], 0.2)  # Simulated lifespan
    df_lifespan_filtered['SIMULATED_MAINTENANCE_TIME'] = np.random.normal(df_lifespan_filtered['MEAN_TIME_MAINTENENCE'], 5)  # Simulated maintenance time
    
    # Simulate the next job start dates based on the random 'SIMULATED_LIFE'
    df_lifespan_filtered['SIMULATED_START_DATE'] = df_lifespan_filtered['EXECUTION_START_DATE'] + pd.to_timedelta(df_lifespan_filtered['SIMULATED_LIFE'], unit='D')
    
    # Calculate the total downtime or total maintenance time for each simulation
    total_maintenance_time = df_lifespan_filtered['SIMULATED_MAINTENANCE_TIME'].sum()
    
    # Store the result of this simulation
    sim_results.append(total_maintenance_time)

# Convert results into a DataFrame
sim_results_df = pd.DataFrame(sim_results, columns=['Total_Maintenance_Time'])

# Analyze the simulation results
optimal_schedule = sim_results_df['Total_Maintenance_Time'].min()  # Find the optimal schedule that minimizes total maintenance time
optimal_simulation = sim_results_df[sim_results_df['Total_Maintenance_Time'] == optimal_schedule].index[0]

print(f"Optimal total maintenance time: {optimal_schedule}")
print(f"Optimal simulation index: {optimal_simulation}")

# Plot the distribution of total maintenance times
plt.figure(figsize=(10, 6))
plt.hist(sim_results_df['Total_Maintenance_Time'], bins=50, alpha=0.7, color='blue')
plt.title('Distribution of Total Maintenance Time (Past Year)')
plt.xlabel('Total Maintenance Time')
plt.ylabel('Frequency')
plt.show()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define the date range (e.g., simulate the past year)
one_year_ago = pd.Timestamp.today() - pd.DateOffset(years=1)

# Filter the DataFrame to include only jobs from the past year
df_lifespan_filtered = df_lifespan[df_lifespan['EXECUTION_START_DATE'] >= one_year_ago].copy()

# Parameters
num_simulations = 10  # Number of Monte Carlo simulations
num_jobs = len(df_lifespan_filtered)  # Number of jobs in the past year
stacking_threshold_days = 30  # Jobs within 30 days will be stacked

# Create a placeholder for the simulation results
sim_results = []
time_saved_results = []

# Monte Carlo simulation loop
for i in range(num_simulations):
    # Simulate the random 'LIFE' and 'MAINTENANCE TIME' for each job in the past year
    df_lifespan_filtered['SIMULATED_LIFE'] = np.random.normal(df_lifespan_filtered['MEAN_LIFESPAN'], 0.2)  # Simulated lifespan
    df_lifespan_filtered['SIMULATED_MAINTENANCE_TIME'] = np.random.normal(df_lifespan_filtered['MEAN_TIME_MAINTENENCE'], 5)  # Simulated maintenance time
    
    # Simulate the next job start dates based on the random 'SIMULATED_LIFE'
    df_lifespan_filtered['SIMULATED_START_DATE'] = df_lifespan_filtered['EXECUTION_START_DATE'] + pd.to_timedelta(df_lifespan_filtered['SIMULATED_LIFE'], unit='D')
    
    # Sort jobs by FUNCTIONAL_LOC and SIMULATED_START_DATE to group jobs in the same location
    df_lifespan_filtered = df_lifespan_filtered.sort_values(['FUNCTIONAL_LOC', 'SIMULATED_START_DATE'])
    
    # Initialize variables to track total maintenance time and time saved
    total_maintenance_time = 0
    time_saved = 0
    
    # Iterate over each job and stack jobs where possible
    for idx, row in df_lifespan_filtered.iterrows():
        # Check for upcoming jobs in the same FUNCTIONAL_LOC within the next 30 days
        future_jobs = df_lifespan_filtered[
            (df_lifespan_filtered['FUNCTIONAL_LOC'] == row['FUNCTIONAL_LOC']) &
            (df_lifespan_filtered['SIMULATED_START_DATE'] > row['SIMULATED_START_DATE']) &
            (df_lifespan_filtered['SIMULATED_START_DATE'] <= row['SIMULATED_START_DATE'] + pd.Timedelta(days=stacking_threshold_days))
        ]
        
        # Stack future jobs into the current job
        if not future_jobs.empty:
            # Combine the maintenance time of all stacked jobs
            stacked_maintenance_time = row['SIMULATED_MAINTENANCE_TIME'] + future_jobs['SIMULATED_MAINTENANCE_TIME'].sum()
            
            # Save the time difference (stacked jobs are done together, saving time)
            time_saved += future_jobs['SIMULATED_MAINTENANCE_TIME'].sum()
            
            # Remove future jobs (as they are now stacked and handled in the current job)
            df_lifespan_filtered = df_lifespan_filtered.drop(future_jobs.index)
            
            # Update total maintenance time for the stacked jobs
            total_maintenance_time += stacked_maintenance_time
        else:
            # No stacking, add the current job's maintenance time
            total_maintenance_time += row['SIMULATED_MAINTENANCE_TIME']
    
    # Store the result of this simulation
    sim_results.append(total_maintenance_time)
    time_saved_results.append(time_saved)

# Convert results into a DataFrame
sim_results_df = pd.DataFrame({
    'Total_Maintenance_Time': sim_results,
    'Time_Saved': time_saved_results
})

# Analyze the simulation results
optimal_schedule = sim_results_df['Total_Maintenance_Time'].min()  # Find the optimal schedule that minimizes total maintenance time
optimal_simulation = sim_results_df[sim_results_df['Total_Maintenance_Time'] == optimal_schedule].index[0]

print(f"Optimal total maintenance time: {optimal_schedule}")
print(f"Optimal simulation index: {optimal_simulation}")
print(f"Time saved in optimal simulation: {sim_results_df.loc[optimal_simulation, 'Time_Saved']}")

# Plot the distribution of total maintenance times and time saved
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(sim_results_df['Total_Maintenance_Time'], bins=50, alpha=0.7, color='blue')
plt.title('Distribution of Total Maintenance Time (Past Year)')
plt.xlabel('Total Maintenance Time')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(sim_results_df['Time_Saved'], bins=50, alpha=0.7, color='green')
plt.title('Distribution of Time Saved by Stacking Jobs')
plt.xlabel('Time Saved (Minutes)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Analyze the simulation results
optimal_schedule = sim_results_df['Total_Maintenance_Time'].min()  # Find the optimal schedule that minimizes total maintenance time

# Ensure there are matching rows for the optimal schedule
optimal_simulation_rows = sim_results_df[sim_results_df['Total_Maintenance_Time'] == optimal_schedule]

# Check if there are any rows matching the optimal schedule
if not optimal_simulation_rows.empty:
    optimal_simulation = optimal_simulation_rows.index[0]  # Get the first index of the matching rows
    print(f"Optimal total maintenance time: {optimal_schedule}")
    print(f"Optimal simulation index: {optimal_simulation}")
    print(f"Time saved in optimal simulation: {sim_results_df.loc[optimal_simulation, 'Time_Saved']}")
else:
    print("No optimal simulation found.")

sim_results_df